[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [2]:
# Import pandas, numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Read dataset with extra params sep='\t', encoding="latin-1"
labeledTrainData = pd.read_csv("labeledTrainData.csv",sep='\t',encoding="latin-1")
labeledTrainData.sample(10)

,id,sentiment,review
12262,249_10,1,I have seen the trailer for this movie several...
9890,9100_7,1,"Spanish films are into a, if not Golden, defin..."
6790,6188_1,0,This is by far the worst adaptation of Jane Ey...
22092,936_1,0,I don't understand people. Why is it that this...
17750,7840_8,1,BORN TO BOOGIE is a real 'find'--though a rock...
6019,888_4,0,This was a movie i could not wait to see! So i...
8716,1431_3,0,!!!!! POSSIBLE SPOILER !!!!!<br /><br />You`d ...
820,12009_3,0,I had a video of the thing. And I think it was...
8109,7620_1,0,I rented this movie because it sounded pretty ...
7632,11597_4,0,"I will stat of with the plot Alice, having sur..."


## 2. Preprocessing

In [13]:
# stop words

[nltk_data] Downloading package stopwords to /home/dks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from collections import Counter
vocab = Counter()

for twit in labeledTrainData.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('the', 287025),
 ('a', 155092),
 ('and', 152651),
 ('of', 142970),
 ('to', 132568),
 ('is', 103227),
 ('in', 85576),
 ('I', 65971),
 ('that', 64555),
 ('this', 57195),
 ('it', 54416),
 ('/><br', 50935),
 ('was', 46697),
 ('as', 42509),
 ('with', 41718),
 ('for', 41067),
 ('but', 33783),
 ('The', 33760),
 ('on', 30765),
 ('movie', 30496)]

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()

for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('I', 65971),
 ('/><br', 50935),
 ('The', 33760),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435),
 ('It', 10950),
 ('really', 10814),
 ('even', 10605),
 ('see', 10154),
 ('-', 9355),
 ('get', 8776),
 ('story', 8523),
 ('much', 8506),
 ('time', 7762),
 ('make', 7485)]

In [20]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Your code here
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [21]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here

In [22]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = labeledTrainData['review']
y = labeledTrainData['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [23]:
X.shape

(25000,)

In [24]:
y.shape

(25000,)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [25]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)# Your code here

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])# Your code here)
clf.fit(X_train, y_train)

D:\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [26]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.8850666666666667
confusion matrix:
 [[3293  503]
 [ 359 3345]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      3796
           1       0.87      0.90      0.89      3704

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



In [31]:
twits = [
    "This is really not bad",
    "I dont love this!",
    ":( :)",
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

This is really not bad --> Negative, Positive = [0.99610592 0.00389408]
I dont love this! --> Negative, Positive = [0.26942903 0.73057097]
:( :) --> Negative, Positive = [0.33408938 0.66591062]


## 5. Export Model 

In [28]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('data', 'Assignment3worklogisticRegression.pkl'), 'wb'), protocol=4)